In [1]:
liked_books = ["4408", "31147619", "29983711", "9401317", "9317691", "8153988", "20494944"]

In [2]:
with open("book_id_map.csv") as f:
    for _ in range(10):
        print(f.readline())

book_id_csv,book_id

0,34684622

1,34536488

2,34017076

3,71730

4,30422361

5,33503613

6,33517540

7,34467031

8,6383669



In [3]:
csv_book_mapping = {}
with open("book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        csv_book_mapping[csv_id] = book_id

In [4]:
csv_book_mapping['0']

'34684622'

In [5]:
overlap_users = set()
with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            continue


        try:
            rating = int(rating)
        except ValueError:
            continue
        
        book_id = csv_book_mapping[csv_id]
        
        if book_id in liked_books and rating >= 4:
                overlap_users.add(user_id)
rec_lines = []

In [6]:
with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            book_id = csv_book_mapping[csv_id]
            rec_lines.append([user_id, book_id, rating])

In [7]:
import pandas as pd


recs = pd.DataFrame(rec_lines, columns=["user_id", "book_id", "rating"])
recs["book_id"] = recs["book_id"].astype(str)

In [8]:
top_recs = recs["book_id"].value_counts().head(10)
top_recs = top_recs.index.values

In [9]:
books_titles = pd.read_json("books_titles.json")
books_titles["book_id"] = books_titles["book_id"].astype(str)
books_titles.head()

,book_id,title,ratings,url,cover_image,mod_title
0,1333909,Good Harbor,10,https://www.goodreads.com/book/show/1333909.Go...,https://s.gr-assets.com/assets/nophoto/book/11...,good harbor
1,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",140,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,the unschooled wizard sun wolf and starhawk 12
2,6066819,Best Friends Forever,51184,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,best friends forever
3,287140,Runic Astrology: Starcraft and Timekeeping in ...,15,https://www.goodreads.com/book/show/287140.Run...,https://images.gr-assets.com/books/1413219371m...,runic astrology starcraft and timekeeping in t...
4,287141,The Aeneid for Boys and Girls,46,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,the aeneid for boys and girls


In [10]:
all_recs = recs["book_id"].value_counts()
all_recs = all_recs.to_frame().reset_index()
all_recs.columns = ["book_id", "book_count"]
all_recs.head(5)

,book_id,book_count
0,2767052,1092
1,29983711,1089
2,2657,1074
3,3,1048
4,4671,1028


In [11]:
all_recs = all_recs.merge(books_titles, how="inner", on="book_id")
all_recs["score"] = all_recs["book_count"] * (all_recs["book_count"] / all_recs["ratings"])
all_recs.sort_values("score", ascending=False).head(10)

,book_id,book_count,title,ratings,url,cover_image,mod_title,score
1,29983711,1089,Pachinko,8161,https://www.goodreads.com/book/show/29983711-p...,https://images.gr-assets.com/books/1462393298m...,pachinko,145.315648
7261,35069544,29,This Will Be My Undoing: Living at the Interse...,8,https://www.goodreads.com/book/show/35069544-t...,https://images.gr-assets.com/books/1502099764m...,this will be my undoing living at the intersec...,105.125000
14752,29749094,15,"Superman (DC Icons, #4)",7,https://www.goodreads.com/book/show/29749094-s...,https://s.gr-assets.com/assets/nophoto/book/11...,superman dc icons 4,32.142857
238,4408,327,East of Eden,3447,https://www.goodreads.com/book/show/4408.East_...,https://images.gr-assets.com/books/1323882457m...,east of eden,31.020888
724,9317691,175,The Name of the Wind (The Kingkiller Chronicle...,1043,https://www.goodreads.com/book/show/9317691-th...,https://images.gr-assets.com/books/1360558233m...,the name of the wind the kingkiller chronicle 1,29.362416
236,32920226,328,"Sing, Unburied, Sing",4592,https://www.goodreads.com/book/show/32920226-s...,https://images.gr-assets.com/books/1499340866m...,sing unburied sing,23.428571
13156,36301023,17,"My Plain Jane (The Lady Janies, #2)",13,https://www.goodreads.com/book/show/36301023-m...,https://images.gr-assets.com/books/1507936746m...,my plain jane the lady janies 2,22.230769
13025,26857046,17,The Invisible Life of Addie La Rue,13,https://www.goodreads.com/book/show/26857046-t...,https://s.gr-assets.com/assets/nophoto/book/11...,the invisible life of addie la rue,22.230769
16306,35430702,14,The Lady's Guide to Petticoats and Piracy,9,https://www.goodreads.com/book/show/35430702-t...,https://s.gr-assets.com/assets/nophoto/book/11...,the ladys guide to petticoats and piracy,21.777778
216,30753987,342,The Leavers,5602,https://www.goodreads.com/book/show/30753987-t...,https://images.gr-assets.com/books/1489158974m...,the leavers,20.878972


In [12]:
popular_recs = all_recs[all_recs["book_count"] > 200].sort_values("score", ascending=False)

In [13]:
def make_clickable(val):
    return '{}">Goodreads'.format(val, val)


def show_image(val):
    return '{}">{}" width=50>'.format(val, val)




popular_recs[~popular_recs["book_id"].isin(liked_books)].head(10).style.format({'url': make_clickable, 'cover_image': show_image})

,book_id,book_count,title,ratings,url,cover_image,mod_title,score
236,32920226,328,"Sing, Unburied, Sing",4592,"https://www.goodreads.com/book/show/32920226-sing-unburied-sing"">Goodreads","https://images.gr-assets.com/books/1499340866m/32920226.jpg"">https://images.gr-assets.com/books/1499340866m/32920226.jpg"" width=50>",sing unburied sing,23.428571
216,30753987,342,The Leavers,5602,"https://www.goodreads.com/book/show/30753987-the-leavers"">Goodreads","https://images.gr-assets.com/books/1489158974m/30753987.jpg"">https://images.gr-assets.com/books/1489158974m/30753987.jpg"" width=50>",the leavers,20.878972
441,33253215,236,The Heart's Invisible Furies,3629,"https://www.goodreads.com/book/show/33253215-the-heart-s-invisible-furies"">Goodreads","https://images.gr-assets.com/books/1490803456m/33253215.jpg"">https://images.gr-assets.com/books/1490803456m/33253215.jpg"" width=50>",the hearts invisible furies,15.347479
71,30688435,533,Exit West,21378,"https://www.goodreads.com/book/show/30688435-exit-west"">Goodreads","https://images.gr-assets.com/books/1477324680m/30688435.jpg"">https://images.gr-assets.com/books/1477324680m/30688435.jpg"" width=50>",exit west,13.288848
251,32283423,321,American War,7776,"https://www.goodreads.com/book/show/32283423-american-war"">Goodreads","https://images.gr-assets.com/books/1481494946m/32283423.jpg"">https://images.gr-assets.com/books/1481494946m/32283423.jpg"" width=50>",american war,13.251157
228,26025588,335,Behold the Dreamers,8793,"https://www.goodreads.com/book/show/26025588-behold-the-dreamers"">Goodreads","https://images.gr-assets.com/books/1439643293m/26025588.jpg"">https://images.gr-assets.com/books/1439643293m/26025588.jpg"" width=50>",behold the dreamers,12.762993
345,17912498,266,The Queen of the Night,6115,"https://www.goodreads.com/book/show/17912498-the-queen-of-the-night"">Goodreads","https://images.gr-assets.com/books/1460425080m/17912498.jpg"">https://images.gr-assets.com/books/1460425080m/17912498.jpg"" width=50>",the queen of the night,11.570891
325,28114515,276,The Wangs vs. the World,7044,"https://www.goodreads.com/book/show/28114515-the-wangs-vs-the-world"">Goodreads","https://images.gr-assets.com/books/1451493647m/28114515.jpg"">https://images.gr-assets.com/books/1451493647m/28114515.jpg"" width=50>",the wangs vs the world,10.814310
108,34273236,459,Little Fires Everywhere,21135,"https://www.goodreads.com/book/show/34273236-little-fires-everywhere"">Goodreads","https://images.gr-assets.com/books/1490351351m/34273236.jpg"">https://images.gr-assets.com/books/1490351351m/34273236.jpg"" width=50>",little fires everywhere,9.968346
104,28815371,467,The Mothers,22346,"https://www.goodreads.com/book/show/28815371-the-mothers"">Goodreads","https://images.gr-assets.com/books/1460652458m/28815371.jpg"">https://images.gr-assets.com/books/1460652458m/28815371.jpg"" width=50>",the mothers,9.759644
